In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
data_train = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')
print(sum(data_train["Hepatomegaly"]==np.nan))
np.sum(data_train.isna())


0


id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64

In [4]:
X = data_train.iloc[:,1:-1]
#print(X_train.head())
y = data_train.iloc[:,-1]

#print(y_train.head())

In [5]:
print(data_train['Hepatomegaly'].unique())

['N' 'Y']


In [6]:

dict_cat = {'Drug' : {'D-penicillamine':0, 'Placebo':1},
        'Sex' : {'M':0,'F':1},
        'Ascites' : {'N':0, 'Y':1},
        'Spiders' : {'N':0, 'Y':1},
        'Hepatomegaly':{'N':0, 'Y':1},
        'Edema' : {'N':0, 'Y':1,'S':2}}


In [7]:
def encode_with_numericals(X, dict_cat):
    for col in dict_cat:#drug, sex...
        print(col)
        for val in dict_cat[col]:#MF,YN...
            X.loc[ X[col] == val , col ] = int(dict_cat[col][val])
            print(X.loc[ X[col] == val , col ])
    return X

In [8]:
X = encode_with_numericals(X, dict_cat)
{'Status':{'D':0, 'C':1,'CL':2}}
y.loc[y=='D'] = 0
y.loc[y=='C'] = 1
y.loc[y=='CL'] = 2
print(y.head())
y=y.astype('int')

Drug
Series([], Name: Drug, dtype: object)
Series([], Name: Drug, dtype: object)
Sex
Series([], Name: Sex, dtype: object)
Series([], Name: Sex, dtype: object)
Ascites
Series([], Name: Ascites, dtype: object)
Series([], Name: Ascites, dtype: object)
Spiders
Series([], Name: Spiders, dtype: object)
Series([], Name: Spiders, dtype: object)
Hepatomegaly
Series([], Name: Hepatomegaly, dtype: object)
Series([], Name: Hepatomegaly, dtype: object)
Edema
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)
0    0
1    1
2    0
3    1
4    1
Name: Status, dtype: object


/tmp/ipykernel_42/2808682700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[y=='D'] = 0
/tmp/ipykernel_42/2808682700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[y=='C'] = 1
/tmp/ipykernel_42/2808682700.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[y=='CL'] = 2


In [9]:
print(sum(y==0))
print(sum(y==1))
print(sum(y==2))

2665
4965
275


In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [11]:
#ss = StandardScaler()
#X_normalized = ss.fit_transform(X)

In [12]:
import seaborn as sns
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=0, stratify=y)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
from sklearn.svm import SVC

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
from sklearn.decomposition import PCA
def make_PCA(X, n_comp = 15):
    pca = PCA(n_components=n_comp)
    return pca.fit_transform(X)

In [19]:
def X_normalize(X_dataset):
    eps = 10^-7
    X_train_cont = X_dataset.drop(dict_cat.keys(), axis=1)
    X_train_normalized = (X_train_cont-X_train_cont.min())/(X_train_cont.max()-X_train_cont.min())\
    .astype(np.float16)
    X_train_normalized = pd.concat((X_train_normalized, X_dataset[dict_cat.keys()].astype(np.int16)), axis=1)
    return X_train_normalized

In [20]:
def X_one_hot_encode(X_dataset):
    ohe = OneHotEncoder(sparse_output=False);
    X_train_cat = X_dataset[dict_cat.keys()]
    encoded_part_train = pd.get_dummies(X_dataset[dict_cat.keys()], columns=dict_cat.keys(),
                                        dtype=np.int16)
    
    X_train_numericals = X_dataset.drop(dict_cat.keys(), axis=1)
    X_train_resampled_encoded = pd.concat((X_train_numericals, encoded_part_train),axis=1)
    return X_train_resampled_encoded

In [21]:
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import RandomUnderSampler
def resample(X_dataset, y_train):
    sm  = SMOTE(k_neighbors=2)
    sm_nc = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = sm.fit_resample(X_dataset, y_train.astype(np.int16))
    return X_train_resampled, y_train_resampled

In [22]:


X_train_normalized = X_normalize(X_train)

X_train_resampled, y_train_resampled = resample(X_train_normalized, y_train)
X_train_resampled_encoded = X_one_hot_encode(X_train_resampled)
X_train_normalized_encoded = X_one_hot_encode(X_train_normalized)
#X_train_resampled_encoded = make_PCA(X_train_resampled_encoded, n_comp = 18)
X_val_normalized = X_normalize(X_val)
X_val_encoded = X_one_hot_encode(X_val_normalized)
#X_val_encoded = make_PCA(X_val_encoded, n_comp = 18)

In [23]:
print(sum(y_train_resampled == 0))

4220


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_text
from sklearn.ensemble import BaggingClassifier


In [25]:
from sklearn.svm import SVC
def use_svm():
    svm = SVC(C=1, kernel='rbf',degree=7)
    svm.fit(X_train_resampled_encoded, y_train_resampled)
    pred_val = svm.predict(X_val_encoded)
    pred_train = svm.predict(X_train_resampled_encoded)
    print('TRAIN:')
    print(accuracy_score(y_train_resampled,pred_train))
    print(f1_score(y_train_resampled,pred_train, average=None))
    print('VALIDATION:')
    print(accuracy_score(y_val,pred_val))
    print(f1_score(y_val,pred_val, average=None))

In [26]:
import lightgbm as lgb 

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [41]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibrationDisplay
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
def use_RandomForest(X_train, y_train, X_val, y_val):
    xgb = RandomForestClassifier()
    params = {'max_depth':[2,4,6,8],'n_estimators':[10,50,100,200]}
    X_train_xgb, X_train_calibrate, y_train_xgb, y_train_calibrate = train_test_split(X_train, y_train, test_size=0.4,
                                                                                      random_state=0, stratify=y_train)
    
    cv = GridSearchCV(xgb, param_grid=params, scoring='neg_log_loss')

    cv.fit(X_train_xgb, y_train_xgb)
    xgb = cv.best_estimator_
    
    calibrated_xgb = CalibratedClassifierCV(xgb, method='sigmoid')
    calibrated_xgb.fit(X_train_calibrate, y_train_calibrate)
    
    pred_train = xgb.predict(X_train_xgb)
    pred_val = xgb.predict(X_val)
    
    print('TRAIN:')
    print(accuracy_score(y_train_xgb,pred_train))
    print(f1_score(y_train_xgb,pred_train, average=None))
    
    print('VALIDATION:')
    print(accuracy_score(y_val,pred_val))
    print(f1_score(y_val,pred_val, average=None))
    
    ll = log_loss(y_val,xgb.predict_proba(X_val))
    l2 = log_loss(y_val,calibrated_xgb.predict_proba(X_val))
    print(ll)
    print(l2)
    return xgb

In [42]:
rf =  use_RandomForest(X_train_normalized_encoded, y_train, X_val_encoded, y_val)

TRAIN:
0.880426693128256
[0.84247375 0.91883786 0.19230769]
VALIDATION:
0.8237774030354131
[0.76463263 0.87696335 0.        ]
0.503899546363843
0.5024890777613589


In [43]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [36]:
from sklearn.metrics import log_loss
from sklearn.calibration import CalibrationDisplay
from sklearn.calibration import CalibratedClassifierCV
def use_lightGBM(X_train, y_train, X_val, y_val):
    xgb = lgb.LGBMClassifier(max_depth=4, learning_rate=0.03, n_estimators=150)
    
    X_train_xgb, X_train_calibrate, y_train_xgb, y_train_calibrate = train_test_split(X_train, y_train, test_size=0.4,
                                                                                      random_state=0, stratify=y_train)
    
    xgb.fit(X_train_xgb, y_train_xgb)
    
    calibrated_xgb = CalibratedClassifierCV(xgb, method='sigmoid')
    calibrated_xgb.fit(X_train_calibrate, y_train_calibrate)
    
    pred_train = xgb.predict(X_train_xgb)
    pred_val = xgb.predict(X_val)
    
    print('TRAIN:')
    print(accuracy_score(y_train_xgb,pred_train))
    print(f1_score(y_train_xgb,pred_train, average=None))
    
    print('VALIDATION:')
    print(accuracy_score(y_val,pred_val))
    print(f1_score(y_val,pred_val, average=None))
    
    ll = log_loss(y_val,xgb.predict_proba(X_val))
    l2 = log_loss(y_val,calibrated_xgb.predict_proba(X_val))
    print(ll)
    print(l2)
    return xgb

In [37]:
xgb = use_lightGBM(X_train_normalized_encoded, y_train, X_val_encoded, y_val)

TRAIN:
0.8652939717191764
[0.81751825 0.90523927 0.36046512]
VALIDATION:
0.815345699831366
[0.756691   0.86702128 0.17391304]
0.4743336091575555
0.5015355628411992


In [30]:
def write_results(model):
    test_data = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')
    id = test_data['id']
    X_test_data =  test_data.drop('id',axis=1)
    X_test_data = encode_with_numericals(X_test_data, dict_cat)
    X_test_data_normalized = X_normalize(X_test_data)
    X_test_data_normalized_encoded = X_one_hot_encode(X_test_data_normalized)
    pred_test = model.predict_proba(X_test_data_normalized_encoded)
    result = pd.DataFrame({'id':id, 'Status_D':pred_test[:,0],'Status_C':pred_test[:,1], 'Status_CL':pred_test[:,2]})
    result.to_csv('submission.csv',index=False);

In [31]:

write_results(xgb)

Drug
Series([], Name: Drug, dtype: object)
Series([], Name: Drug, dtype: object)
Sex
Series([], Name: Sex, dtype: object)
Series([], Name: Sex, dtype: object)
Ascites
Series([], Name: Ascites, dtype: object)
Series([], Name: Ascites, dtype: object)
Spiders
Series([], Name: Spiders, dtype: object)
Series([], Name: Spiders, dtype: object)
Hepatomegaly
Series([], Name: Hepatomegaly, dtype: object)
Series([], Name: Hepatomegaly, dtype: object)
Edema
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)


In [127]:
import matplotlib.pyplot as plt
def plot_feature_hists(X, y):
    common_params = dict(bins=10)
    common_params['histtype'] = 'bar'
    a = X[y == 0]
    b = X[y == 1]
    c = X[y == 2]
    fig, ax = plt.subplots(nrows=len(a.columns), ncols=1,figsize=(15,50) )
    colors = ['Status_D', 'Status_C', 'Status_CL']
    for idx, col in enumerate(a):
        ax[idx].hist((a[col], b[col], c[col]),label = colors, **common_params )
        ax[idx].legend()
    plt.show()

In [ ]:
plot_feature_hists(X_train_resampled_encoded, y_train_resampled)
#[(y_train == 0) | (y_train == 1)]

In [46]:
from sklearn.ensemble import StackingClassifier

In [77]:

def use_StackingClassifier(estimators, final_estimator, X_train, y_train, X_val, y_val):
    sc = StackingClassifier(estimators,final_estimator)
    sc.fit( X_train, y_train)
    
    calibrated_sc = CalibratedClassifierCV(sc, method='sigmoid')
    calibrated_sc.fit(X_train, y_train)
    
    pred_train = sc.predict(X_train)
    pred_train_proba = sc.predict_proba(X_train)
    
    pred_val = sc.predict(X_val)
    pred_proba_val = sc.predict_proba(X_val)
    
    
    pred_proba_val_calibrated = calibrated_sc.predict_proba(X_val)
    
    l1 = log_loss(y_train, pred_train_proba)
    l2 = log_loss(y_val, pred_proba_val)
    l3 = log_loss(y_val, pred_proba_val_calibrated)
    print(l1)
    print(l2)
    print(l3)
    return sc, calibrated_sc

In [78]:
light_gb = lgb.LGBMClassifier(max_depth=4, learning_rate=0.03, n_estimators=150)
random_forest = RandomForestClassifier(max_depth=7, n_estimators=150)

stacking_clf, calibrated_stacking_clf = use_StackingClassifier([['lgbm',light_gb],['rf',random_forest]], None, X_train_normalized_encoded, y_train,
                       X_val_encoded,y_val)

0.37910113293998
0.48659823235889554
0.47730293395843326


In [79]:
write_results(calibrated_stacking_clf)

Drug
Series([], Name: Drug, dtype: object)
Series([], Name: Drug, dtype: object)
Sex
Series([], Name: Sex, dtype: object)
Series([], Name: Sex, dtype: object)
Ascites
Series([], Name: Ascites, dtype: object)
Series([], Name: Ascites, dtype: object)
Spiders
Series([], Name: Spiders, dtype: object)
Series([], Name: Spiders, dtype: object)
Hepatomegaly
Series([], Name: Hepatomegaly, dtype: object)
Series([], Name: Hepatomegaly, dtype: object)
Edema
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)
Series([], Name: Edema, dtype: object)
